# 3-3,高阶API示范

下面的范例使用TensorFlow的高阶API实现线性回归模型。

TensorFlow的高阶API主要为tf.keras.models提供的模型的类接口。


使用Keras接口有以下3种方式构建模型：使用Sequential按层顺序构建模型，使用函数式API构建任意结构模型，继承Model基类构建自定义模型。

此处分别演示使用Sequential按层顺序构建模型以及继承Model基类构建自定义模型。

### 一，使用Sequential按层顺序构建模型【面向新手】

In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers

#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

In [2]:
tf.keras.backend.clear_session()

linear = models.Sequential()
linear.add(layers.Dense(1,input_shape =(2,)))
linear.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [3]:
### 使用fit方法进行训练

linear.compile(optimizer="adam",loss="mse",metrics=["mae"])
linear.fit(X,Y,batch_size = 20,epochs = 200)  

tf.print("w = ",linear.layers[0].kernel)
tf.print("b = ",linear.layers[0].bias)


Train on 800 samples
Epoch 1/200
800/800 [==============================] - 0s 431us/sample - loss: 59.5923 - mae: 6.4031
Epoch 2/200
800/800 [==============================] - 0s 74us/sample - loss: 55.7390 - mae: 6.1912
Epoch 3/200
800/800 [==============================] - 0s 75us/sample - loss: 52.1282 - mae: 5.9840
Epoch 4/200
800/800 [==============================] - 0s 77us/sample - loss: 48.7683 - mae: 5.7867
Epoch 5/200
800/800 [==============================] - 0s 79us/sample - loss: 45.6057 - mae: 5.5918
Epoch 6/200
800/800 [==============================] - 0s 80us/sample - loss: 42.6979 - mae: 5.4078
Epoch 7/200
800/800 [==============================] - 0s 72us/sample - loss: 39.9568 - mae: 5.2268
Epoch 8/200
800/800 [==============================] - 0s 77us/sample - loss: 37.4595 - mae: 5.0529
Epoch 9/200
800/800 [==============================] - 0s 75us/sample - loss: 35.1030 - mae: 4.8895
Epoch 10/200
800/800 [==============================] - 0s 85us/sample - loss:

800/800 [==============================] - 0s 71us/sample - loss: 4.4371 - mae: 1.6896
Epoch 82/200
800/800 [==============================] - 0s 72us/sample - loss: 4.4143 - mae: 1.6850
Epoch 83/200
800/800 [==============================] - 0s 81us/sample - loss: 4.3931 - mae: 1.6808
Epoch 84/200
800/800 [==============================] - 0s 69us/sample - loss: 4.3719 - mae: 1.6768
Epoch 85/200
800/800 [==============================] - 0s 79us/sample - loss: 4.3543 - mae: 1.6733
Epoch 86/200
800/800 [==============================] - 0s 81us/sample - loss: 4.3356 - mae: 1.6693
Epoch 87/200
800/800 [==============================] - 0s 80us/sample - loss: 4.3165 - mae: 1.6652
Epoch 88/200
800/800 [==============================] - 0s 71us/sample - loss: 4.3008 - mae: 1.6621
Epoch 89/200
800/800 [==============================] - 0s 77us/sample - loss: 4.2871 - mae: 1.6596
Epoch 90/200
800/800 [==============================] - 0s 74us/sample - loss: 4.2695 - mae: 1.6559
Epoch 91/200


800/800 [==============================] - 0s 95us/sample - loss: 4.0646 - mae: 1.6138
Epoch 163/200
800/800 [==============================] - 0s 72us/sample - loss: 4.0677 - mae: 1.6152
Epoch 164/200
800/800 [==============================] - 0s 64us/sample - loss: 4.0661 - mae: 1.6139
Epoch 165/200
800/800 [==============================] - 0s 62us/sample - loss: 4.0660 - mae: 1.6147
Epoch 166/200
800/800 [==============================] - 0s 66us/sample - loss: 4.0659 - mae: 1.6138
Epoch 167/200
800/800 [==============================] - 0s 82us/sample - loss: 4.0671 - mae: 1.6142
Epoch 168/200
800/800 [==============================] - 0s 82us/sample - loss: 4.0648 - mae: 1.6139
Epoch 169/200
800/800 [==============================] - 0s 85us/sample - loss: 4.0667 - mae: 1.6140
Epoch 170/200
800/800 [==============================] - 0s 82us/sample - loss: 4.0666 - mae: 1.6141
Epoch 171/200
800/800 [==============================] - 0s 65us/sample - loss: 4.0653 - mae: 1.6141
Epoc

### 二，继承Model基类构建自定义模型【面向专家】

In [4]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,losses,metrics


#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [5]:
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

ds_train = tf.data.Dataset.from_tensor_slices((X[0:n*3//4,:],Y[0:n*3//4,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()

ds_valid = tf.data.Dataset.from_tensor_slices((X[n*3//4:,:],Y[n*3//4:,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()


In [6]:
tf.keras.backend.clear_session()

class MyModel(models.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
    def build(self,input_shape):
        self.dense1 = layers.Dense(1)   
        super(MyModel,self).build(input_shape)
    
    def call(self, x):
        y = self.dense1(x)
        return(y)

model = MyModel()
model.build(input_shape =(None,2))
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [7]:
### 自定义训练循环(专家教程)


optimizer = optimizers.Adam()
loss_func = losses.MeanSquaredError()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_metric = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_metric = tf.keras.metrics.MeanAbsoluteError(name='valid_mae')


@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)

@tf.function
def valid_step(model, features, labels):
    predictions = model(features)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)
    

@tf.function
def train_model(model,ds_train,ds_valid,epochs):
    for epoch in tf.range(1,epochs+1):
        for features, labels in ds_train:
            train_step(model,features,labels)

        for features, labels in ds_valid:
            valid_step(model,features,labels)

        logs = 'Epoch={},Loss:{},MAE:{},Valid Loss:{},Valid MAE:{}'
        
        if  epoch%100 ==0:
            printbar()
            tf.print(tf.strings.format(logs,
            (epoch,train_loss.result(),train_metric.result(),valid_loss.result(),valid_metric.result())))
            tf.print("w=",model.layers[0].kernel)
            tf.print("b=",model.layers[0].bias)
            tf.print("")
        
        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

train_model(model,ds_train,ds_valid,400)


================================================================================16:27:08
Epoch=100,Loss:104.5952,MAE:8.19842052,Valid Loss:118.657539,Valid MAE:8.79173183
w= [[1.36984992]
 [-0.972368121]]
b= [1.38565648]

================================================================================16:27:15
Epoch=200,Loss:55.5088463,MAE:5.11994123,Valid Loss:62.7634163,Valid MAE:5.44431162
w= [[1.98663604]
 [-0.996249795]]
b= [2.77318311]

================================================================================16:27:21
Epoch=300,Loss:38.301754,MAE:3.95671964,Valid Loss:43.0225525,Valid MAE:4.14339161
w= [[1.99494064]
 [-0.997208714]]
b= [2.97018743]

================================================================================16:27:28
Epoch=400,Loss:29.7181396,MAE:3.37643766,Valid Loss:33.1717262,Valid MAE:3.49341393
w= [[1.99479365]
 [-0.997226357]]
b= [2.97108555]

